# Big Data Dashboarding
---

In this notebook we are going to put our visualizations together into a dashboard

We will also introduce a new tool from the HoloViz suite:

<img src="images/panel_logo.png" width="100">

Panel is a high-level app and dashboarding solution for Python

## Lets reconnect to our Dask Cluster and load our dataset 

In [ ]:
import param
import panel as pn

pn.extension()

In [ ]:
import dask_gateway
import dask.dataframe as dd
gateway = dask_gateway.Gateway()

In [ ]:
class LaunchDaskCluster(param.Parameterized):

    workers = pn.widgets.IntRangeSlider(name='Number of Workers', start=1, end=10, value=(1, 5), step=1)
    
    launch_dask = pn.widgets.Button(name='Launch Dask')
    dask_status = pn.widgets.StaticText(name='Dask Status', value='Not Connected')
    cluster = None
    client = None
    ready = param.Boolean(default=False, precedence=-1)

    def output(self):
        return self.client

    @launch_dask.on_click
    def dask_launcher(self):
        if len(running_clusters := gateway.list_clusters())>0:
            #self.dask_status.value = "Found existing dask cluster, connecting and rescaling"
            self.cluster = gateway.connect(running_clusters[0].name)
            self.cluster.adapt(self.min_workers, self.max_workers)
        else:
            #self.dask_status.value = "Launching new dask cluster"
            self.cluster = gateway.new_cluster(conda_environment="pycon2023/pycon2023-tutorial", profile="Medium Worker")
            self.cluster.adapt(self.workers.value[0], self.workers.value[1])
        self.client = self.cluster.get_client()
        #self.dask_status.value = "Waiting for at least 1 worker"
        #client.wait_for_workers(1)
        #self.dask_status.value = "Cluster Ready"
        self.ready.value = True

    def panel(self):
        return pn.Column(
            self.workers, 
            self.launch_dask,
            self.dask_status,
        )

stage1 = LaunchDaskCluster()
stage1.panel()

In [ ]:
class Dashboard(param.Parameterized):
    years = pn.widgets.IntRangeSlider(name='Years', start=2003, end=2022, value=(2018, 2022), step=1)
    flights = []
    
    @depends(years)
    def read_data(self):
        self.flights = 
    

In [ ]:
# from kim
class LaunchDaskCluster(param.Parameterized):

    workers = pn.widgets.IntRangeSlider(name='Number of Workers', start=1, end=10, value=(1, 5), step=1)
    
    launch_dask = pn.widgets.Button(name='Launch Dask')
    dask_status = pn.widgets.StaticText(name='Dask Status', value='Not Connected')
    cluster = None
    client = None
    ready = param.Boolean(default=False, precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.launch_dask.on_click(self.dask_launcher)

    def output(self):
        return self.client

    # @launch_dask.on_click
    
    def dask_launcher(self, event=None):
        if len(running_clusters := gateway.list_clusters())>0:
            #self.dask_status.value = "Found existing dask cluster, connecting and rescaling"
            self.cluster = gateway.connect(running_clusters[0].name)
            self.cluster.adapt(self.workers.value[0], self.workers.value[1])
        else:
            #self.dask_status.value = "Launching new dask cluster"
            self.cluster = gateway.new_cluster(conda_environment="pycon2023/pycon2023-tutorial", profile="Medium Worker")
            self.cluster.adapt(self.workers.value[0], self.workers.value[1])
        self.client = self.cluster.get_client()
        #self.dask_status.value = "Waiting for at least 1 worker"
        #client.wait_for_workers(1)
        #self.dask_status.value = "Cluster Ready"
        self.ready = True

    def panel(self):
        return pn.Column(
            self.workers, 
            self.launch_dask,
            self.dask_status,
        )